In [1]:
# data analysis imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# NLP Imports
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import wordninja

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ayaanhaque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayaanhaque/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df = pd.read_csv('../data/bad.csv')

In [21]:
df.head()

,title,selftext,author,num_comments,is_suicide,url,selftext_clean,title_clean,author_clean
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,133,0,https://www.reddit.com/r/depression/comments/d...,understand people reply immediately op invitat...,broken least understood rule helper may invite...,sql witch
1,Regular Check-In Post,Welcome to /r/depression's check-in post - a p...,circinia,1644,0,https://www.reddit.com/r/depression/comments/e...,welcome r depression check post place take mom...,regular check post,c irc
2,I hate it so much when you try and express you...,I've been feeling really depressed and lonely ...,TheNewKiller69,8,0,https://www.reddit.com/r/depression/comments/f...,feeling really depressed lonely lately job ful...,hate much try express feeling parent turn arou...,new killer 69
3,I went to the hospital because I was having re...,I literally broke down crying and asked to go ...,Jazzlecrab,15,0,https://www.reddit.com/r/depression/comments/f...,literally broke cry asked go home another nurs...,went hospital wa really bad panic attack theyc...,jazz le crab
4,Cake day for me,Any kind soul want to give a depressed person ...,Depressed_Kid786,37,0,https://www.reddit.com/r/depression/comments/f...,kind soul want give depressed person karma edi...,cake day,depressed kid 786


In [23]:
for row in df.index:
    if df["is_suicide"][row] == 1:
        df["is_suicide"][row] = 0
    else:
        df["is_suicide"][row] = 1

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
df.head()

,title,selftext,author,num_comments,is_suicide,url,selftext_clean,title_clean,author_clean
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,133,1,https://www.reddit.com/r/depression/comments/d...,understand people reply immediately op invitat...,broken least understood rule helper may invite...,sql witch
1,Regular Check-In Post,Welcome to /r/depression's check-in post - a p...,circinia,1644,1,https://www.reddit.com/r/depression/comments/e...,welcome r depression check post place take mom...,regular check post,c irc
2,I hate it so much when you try and express you...,I've been feeling really depressed and lonely ...,TheNewKiller69,8,1,https://www.reddit.com/r/depression/comments/f...,feeling really depressed lonely lately job ful...,hate much try express feeling parent turn arou...,new killer 69
3,I went to the hospital because I was having re...,I literally broke down crying and asked to go ...,Jazzlecrab,15,1,https://www.reddit.com/r/depression/comments/f...,literally broke cry asked go home another nurs...,went hospital wa really bad panic attack theyc...,jazz le crab
4,Cake day for me,Any kind soul want to give a depressed person ...,Depressed_Kid786,37,1,https://www.reddit.com/r/depression/comments/f...,kind soul want give depressed person karma edi...,cake day,depressed kid 786


In [25]:
df.tail()

,title,selftext,author,num_comments,is_suicide,url,selftext_clean,title_clean,author_clean
1748,"As a male, I’m so tired of the lack of unique ...",As a male it’s insanely frustrating to browse ...,Childish_Brandino,2416,0,https://www.reddit.com/r/CasualConversation/co...,male insanely frustrating browse woman clothin...,male tired lack unique clothing available,childish brandi
1749,My teenager made me so proud tonight with a si...,I have 4 boys ranging from ages 3 to 14. Tonig...,SedativeCorpse,251,0,https://www.reddit.com/r/CasualConversation/co...,4 boy ranging age 3 14 tonight 3 year old wa t...,teenager made proud tonight simple gesture,sedative corpse
1750,"After 30 years of being open, my family’s rest...",My family has owned a fine dining italian rest...,retirereddit,769,0,https://www.reddit.com/r/CasualConversation/co...,family ha owned fine dining italian restaurant...,30 year open family restaurant closing tonight,retire reddit
1751,This coronavirus things has made me realize pe...,Just live streams of people at home getting in...,ghost_sanctum,2996,0,https://www.reddit.com/r/CasualConversation/co...,live stream people home getting touch instrume...,coronavirus thing ha made realize people would...,ghost sanctum
1752,I’m a lunch lady at an elementary school schoo...,At first I was mad that I would still have to ...,befuddledghostie,1695,0,https://www.reddit.com/r/CasualConversation/co...,first wa mad would still work everyone else ge...,lunch lady elementary school school cancelled ...,befuddled ghost ie


In [26]:
df["is_suicide"].sum()

917

In [27]:
df.to_csv('../data/depression_vs_nothing_final.csv', index = False)